In [6]:
import os.path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from pathlib import Path
from tqdm import tqdm
from time import perf_counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

import efficientnet.keras as efn
from keras import backend as K

In [2]:
dir_ = Path('./final_test_brand')
filepaths = list(dir_.glob(r'**/*'))
def proc_img(filepath):
    """
   		이미지데이터의 경로와 label데이터로 데이터프레임 만들기 
    """

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # 경로와 라벨 concatenate
    test_df3 = pd.concat([filepath, labels], axis=1)

    # index 재설정
    test_df3 = test_df3.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return test_df3

test_df3 = proc_img(filepaths)
test_df3.head(5)

,Filepath,Label
0,final_test_brand/twosome/KakaoTalk_Photo_2022-...,twosome
1,final_test_brand/hollys/hollys_good (229).jpg,hollys
2,final_test_brand/hollys/hollys_good (263).jpg,hollys
3,final_test_brand/twosome/KakaoTalk_Photo_2022-...,twosome
4,final_test_brand/ediya/IMG_6179.jpg,ediya


In [3]:
spac = test_df3[test_df3['Label'].str.contains('final_test_brand')].index

test_df3.drop(spac, inplace=True)

print(f'Number of pictures: {test_df3.shape[0]}\n')
print(f'Number of different labels: {len(test_df3.Label.unique())}\n')
print(f'Labels: {test_df3.Label.unique()}')

Number of pictures: 31

Number of different labels: 4

Labels: ['twosome' 'hollys' 'ediya' 'starbucks']


In [4]:
def create_gen():
    # 생성기 및 데이터 증강으로 이미지 로드
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.1
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df3,
        x_col='Filepath',
        y_col='Label',
        target_size=(448, 448),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    
    return test_generator,test_images

In [5]:
from tensorflow.keras.models   import Sequential , clone_model , Model, load_model

In [35]:
test_images.class_indices

{'ediya': 0, 'hollys': 1, 'starbucks': 2, 'twosome': 3}

In [22]:
dir_ = Path('./final_models/brand')
model_filepaths = list(dir_.glob(r'**/*.h5'))
def proc_img2(model_filepath):
    """
    """

#     labels = [str(filepath[i]).split("/")[-2] \
#               for i in range(len(filepath))]

    model_filepath = pd.Series(model_filepath, name='Filepath').astype(str)
#     labels = pd.Series(labels, name='Label')

    # 경로와 라벨 concatenate
    model_df = pd.DataFrame(model_filepath)
#     test_df3 = pd.concat([filepath, labels], axis=1)

    # index 재설정
    model_df = model_df.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return model_df

model_df = proc_img2(model_filepaths)
model_df


,Filepath
0,final_models/brand/brand_recog_model.h5
1,final_models/brand/brand_recog_model_0622_mbn.h5
2,final_models/brand/brand_recog_model_0622_mbn3.h5
3,final_models/brand/brand_v3_adam_epoch20_batch...
4,final_models/brand/brand_Xception_adam_epoch20...
5,final_models/brand/brand_recog_model_0622_mbn2.h5
6,final_models/brand/brand_v3_adam1.h5
7,final_models/brand/brand_shufflenet_adam_epoch...


In [23]:
model_list = model_df.values.tolist()

In [24]:
print('2번 모델 복원')
# loaded_model = tf.keras.models.load_model('brand_shufflenet_adam_epoch20_batch16.h5')
for i in model_list:
    print(i[0])
    mod_path = './'+i[0]
    loaded_model = tf.keras.models.load_model(mod_path, custom_objects={'tf': tf})
#     loaded_model.summary()

    test_generator,test_images=create_gen()



    test_images.class_indices

    # Predict the label of the test_images
    pred = loaded_model.predict(test_images)
    pred = np.argmax(pred,axis=1)

    # Map the label
    labels = (test_images.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    pred = [labels[k] for k in pred]

    y_test = list(test_df3.Label)
    acc = accuracy_score(y_test,pred)
    print(f'Accuracy on the test set: {acc * 100:.2f}%')

    class_report = classification_report(y_test, pred, zero_division=1)
    print(class_report)

2번 모델 복원
final_models/brand/brand_recog_model.h5
Found 31 validated image filenames belonging to 4 classes.


2022-06-27 11:04:54.692610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 16.13%
              precision    recall  f1-score   support

       ediya       0.00      0.00      0.00         7
      hollys       1.00      0.00      0.00        11
   starbucks       0.17      1.00      0.29         5
     twosome       1.00      0.00      0.00         8

    accuracy                           0.16        31
   macro avg       0.54      0.25      0.07        31
weighted avg       0.64      0.16      0.05        31

final_models/brand/brand_recog_model_0622_mbn.h5
Found 31 validated image filenames belonging to 4 classes.


2022-06-27 11:05:03.005236: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 74.19%
              precision    recall  f1-score   support

       ediya       0.83      0.71      0.77         7
      hollys       1.00      1.00      1.00        11
   starbucks       0.42      1.00      0.59         5
     twosome       1.00      0.25      0.40         8

    accuracy                           0.74        31
   macro avg       0.81      0.74      0.69        31
weighted avg       0.87      0.74      0.73        31

final_models/brand/brand_recog_model_0622_mbn3.h5
Found 31 validated image filenames belonging to 4 classes.


...

Accuracy on the test set: 74.19%
              precision    recall  f1-score   support

       ediya       0.83      0.71      0.77         7
      hollys       1.00      1.00      1.00        11
   starbucks       0.42      1.00      0.59         5
     twosome       1.00      0.25      0.40         8

    accuracy                           0.74        31
   macro avg       0.81      0.74      0.69        31
weighted avg       0.87      0.74      0.73        31

final_models/brand/brand_v3_adam1.h5
Found 31 validated image filenames belonging to 4 classes.


2022-06-27 11:05:29.499959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 58.06%
              precision    recall  f1-score   support

       ediya       0.36      0.71      0.48         7
      hollys       1.00      1.00      1.00        11
   starbucks       0.00      0.00      0.00         5
     twosome       1.00      0.25      0.40         8

    accuracy                           0.58        31
   macro avg       0.59      0.49      0.47        31
weighted avg       0.69      0.58      0.57        31

final_models/brand/brand_shufflenet_adam_epoch20_batch16.h5
Found 31 validated image filenames belonging to 4 classes.


2022-06-27 11:05:36.057206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 67.74%
              precision    recall  f1-score   support

       ediya       0.43      0.43      0.43         7
      hollys       1.00      1.00      1.00        11
   starbucks       0.57      0.80      0.67         5
     twosome       0.50      0.38      0.43         8

    accuracy                           0.68        31
   macro avg       0.62      0.65      0.63        31
weighted avg       0.67      0.68      0.67        31

